## Usando o pca sobre todo o conjunto

In [26]:
import os
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import exposure
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def load_images_from_folder(base_folder, n_components=150):
    images = []
    labels = []
    
    for label in ["plastic", "no-plastic"]:
        folder = os.path.join(base_folder, label)
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            
            # resize e normalização
            img = cv2.resize(img, (256,256))
            img = img.astype("float32") / 255.0
            
            # flatten
            img_flat = img.flatten()
            images.append(img_flat)
            labels.append(1 if label == "plastic" else 0)
    
    X = np.array(images) 
    y = np.array(labels)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
  
    pca = PCA(n_components=n_components, random_state=42)
    X_pca = pca.fit_transform(X_scaled) 
    
    return X_pca, y, scaler, pca

## Extraindo com hog

In [ ]:
def load_images_with_hog(base_folder,
                         pixels_per_cell=(8,8),
                         cells_per_block=(2,2),
                         orientations=9,
                         use_pca=False,
                         n_components=150):
    hog_list = []
    labels   = []

    for label in ["plastic", "no-plastic"]:
        folder = os.path.join(base_folder, label)
        for fname in os.listdir(folder):
            path = os.path.join(folder, fname)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue

            
            img = cv2.resize(img, (256,256)).astype("float32") / 255.0

            
            features = hog(
                img,
                orientations=orientations,
                pixels_per_cell=pixels_per_cell,
                cells_per_block=cells_per_block,
                block_norm='L2-Hys',
                feature_vector=True
            )
            hog_list.append(features)
            labels.append(1 if label=="plastic" else 0)

    X_hog = np.array(hog_list)  
    y     = np.array(labels)

    if not use_pca:
        return X_hog, y

    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_hog)

    pca = PCA(n_components=n_components, random_state=42)
    X_pca = pca.fit_transform(X_scaled)

    return X_pca, y, scaler, pca

In [29]:

X_train, y_train= load_images_with_hog("SOUVIK/train")
X_test, y_test = load_images_with_hog("SOUVIK/test")

In [30]:
print(f"Treino: {X_train.shape}, Rótulos: {y_train.shape}")
print(f"Teste: {X_test.shape}, Rótulos: {y_test.shape}")

Treino: (1720, 34596), Rótulos: (1720,)
Teste: (430, 34596), Rótulos: (430,)


In [58]:
knn = KNeighborsClassifier(n_neighbors=12)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=12)

In [59]:
y_pred = knn.predict(X_test)
class_rep = classification_report(y_test, y_pred)
print(f"Classification Report")
print(class_rep)

cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix")
print(cm)
print(f"TN: {cm[0][0]} | FN: {cm[0][1]} | FP: {cm[1][0]} | TP: {cm[1][1]}")

Classification Report
              precision    recall  f1-score   support

           0       0.59      0.83      0.69       220
           1       0.69      0.39      0.50       210

    accuracy                           0.62       430
   macro avg       0.64      0.61      0.59       430
weighted avg       0.64      0.62      0.60       430

Confusion Matrix
[[183  37]
 [128  82]]
TN: 183 | FN: 37 | FP: 128 | TP: 82


## Teste com SVM

In [60]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf')
svc.fit(X_train, y_train)

SVC()

### SVC com kernel rbf

In [62]:
y_pred_svc = svc.predict(X_test)
class_rep = classification_report(y_test, y_pred_svc)
print(f"Classification Report")
print(class_rep)

cm = confusion_matrix(y_test, y_pred_svc)
print(f"Confusion Matrix")
print(cm)
print(f"TN: {cm[0][0]} | FN: {cm[0][1]} | FP: {cm[1][0]} | TP: {cm[1][1]}")

Classification Report
              precision    recall  f1-score   support

           0       0.82      0.66      0.73       220
           1       0.71      0.85      0.77       210

    accuracy                           0.75       430
   macro avg       0.76      0.76      0.75       430
weighted avg       0.76      0.75      0.75       430

Confusion Matrix
[[146  74]
 [ 32 178]]
TN: 146 | FN: 74 | FP: 32 | TP: 178


USANDO ÁRVORE

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier(max_depth=2)

In [ ]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=2)

In [ ]:
y_pred = tree.predict(X_test)
print("Acurácia:", classification_report(y_test, y_pred))

Acurácia:               precision    recall  f1-score   support

           0       0.59      0.47      0.53       220
           1       0.55      0.66      0.60       210

    accuracy                           0.57       430
   macro avg       0.57      0.57      0.56       430
weighted avg       0.57      0.57      0.56       430



INDIVIDUAL TEST. CHANGE THE PATH TO INPUT IMAGES

0 IS NO_PLASTIC AND 1 IS PLASTIC

In [ ]:
import pickle

# Salve o modelo em um arquivo
filename = 'knn_model.pkl'
pickle.dump(knn, open(filename, 'wb'))

print(f"Modelo KNN exportado para {filename}")

Modelo KNN exportado para knn_model.pkl
